# Controlador JSD Evolution 

Para permitir a evolução do esquema JSON com base nos novos documentos inseridos na coleção, foi desenvolvido o Controlador JSD Evolution. Para se aproximar do cenário idealizado pela evolução incremental do esquema, foi necessário desenvolver um módulo adicional e também são necessárias algumas configurações por parte dos usuários (e.g. administradores do sistema, desenvolvedores de software e analistas de dados). Para isso, foi criado o Controlador JSD Evolution, que monitora a adição de novos documentos à coleção, juntamente com um arquivo de configurações de entrada.

A abordagem JSD Evolution é ativada quando um número específico de novos documentos é adicionado à coleção. O Controlador monitora essas inserções com base em um limite definido pelo administrador. Dessa forma, o administrador tem a prerrogativa de determinar quantos novos documentos são necessários para que o esquema evolua conforme o contexto específico.

![Controlador: Usado para monitorar a coleção](imagem/controlador_2.jpg)

A figura ilustra o funcionamento do Controlador JSD Evolution. Esse processo começa com a recepção das informações necessárias para o monitoramento (caixa de entrada). O controlador responsável (Caixa Controlador JSD Evolution) monitora a adição de novos documentos e a abordagem JSD Evolution. O processo de monitoramento inicia-se com a inserção das informações no arquivo de configuração. A próxima atividade é o monitoramento do conjunto de dados. Além de monitorar as novas inserções no conjunto de dados, o controlador é responsável por acionar a evolução incremental do esquema (caixa JSD Evolution) quando o limiar (l) predefinido pelo usuário administrador for menor ou igual à quantidade de novos documentos (qn). Se qn < l, o controlador continua monitorando a coleção. Após a finalização da evolução incremental do esquema, o fluxo de execução retorna ao Controlador JSD Evolution.


#### Change Streams 

O monitoramento do banco de dados é realizado utilizando a função Change Streams, uma funcionalidade que permite capturar alterações em tempo real nos dados. Para implementar, foi necessário configurar replica sets no banco de dados. Replica sets são conjuntos de instâncias de bancos de dados que mantêm cópias idênticas dos dados para garantir alta disponibilidade e tolerância a falhas. Essa configuração permite que o Change Streams monitore continuamente as mudanças nos dados, fornecendo uma visão em tempo real das atividades no banco de dados.


* [Replication](https://www.mongodb.com/docs/manual/replication/)
* [Deploy a Replica Set](https://www.mongodb.com/docs/manual/tutorial/deploy-replica-set/)


Monitoramento do Banco de dados MongoDB usando Replica Set

[MongoDB Change Streams with Python](https://www.mongodb.com/developer/languages/python/python-change-streams/)




#### Imports

In [ ]:
import requests
import pymongo
import time
import logging
import csv
import json
from bson.json_util import dumps
from json.decoder import JSONDecodeError

#### Entrada 

In [ ]:
# Leitura do arquivo JSON de configuração
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

databaseName = config['database']['databaseName']
collectionName = config['database']['collectionName']
limite_documentos = config['database']['limite_documentos']
colecao_monitorada = config['database']['colecao_monitorada']
port = config['database']['port']


# token de autenticação 
headers = {
    'Authorization': 'Bearer '+ token,
    'Content-Type': 'application/json'
}

### Criar uma conta

In [ ]:
# Dados do usuário
user_data = {
    'username': config['account']['username'],
    'email': config['account']['email'],
    'password': config['account']['password']
}

# URL requisição
url = 'http://localhost:4200/api/register'

# Fazendo a requisição POST
response = requests.post(url, json=user_data)

# Verificando a resposta
if response.status_code == 200:
    print('Requisição bem-sucedida!')
    print('Resposta:', response.json()) 
    
elif response.status_code == 400:
    try:
        error_details = response.json()
        # Verifica se o erro é de duplicação de email
        if error_details.get('error', {}).get('code') == 11000 and 'email' in error_details.get('error', {}).get('keyPattern', {}):
            print('O email já está registrado.')
        else:
            print('Erro na requisição:', response.status_code)
            print('Detalhes do erro:', response.text)
    except ValueError:
        # Caso a resposta não seja um JSON válido
        print('Erro na requisição:', response.status_code)
        print('Detalhes do erro:', response.text)
else:
    print('Erro na requisição:', response.status_code)
    print('Detalhes do erro:', response.text)


### Acessar a conta criada e recuperar o token de autentificação

In [ ]:
# Dados de login
login_data = {
    'email': config['account']['email'],
    'password': config['account']['password']
}

# URL requisição
login_url = 'http://localhost:4200/api/login' 

# Fazendo a requisição de login
response = requests.post(login_url, json=login_data)

# Verificando a resposta
if response.status_code == 200:
    print('Login bem-sucedido!')
    token = response.json().get('token')
    if token:
        print('Token de acesso:', token)
    else:
        print('Token de acesso não encontrado na resposta.')
else:
    print('Erro no login:', response.json().get('message'))
    print('Código de erro:', response.json().get('code'))


# Monitoramento do banco de dados

#### Funções 

In [ ]:
# Configuração do logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Função para se conectar ao MongoDB
def conectar_mongo():
    return pymongo.MongoClient('mongodb://localhost:27016,localhost:27018,localhost:27019/?replicaSet=rs0')

# Função para iniciar o changestream em uma coleção do MongoDB
def iniciar_changestream(client, collection_name, resume_token=None):
    return client.changestream[collection_name].watch(
        [{'$match': {'operationType': {'$in': ['insert']}}}],
        resume_after=resume_token
    )

#Função para salvar um documento em outro banco de dados MongoDB
def salvar_documento_banco_temporario(documento, colecao_temporaria):
    try:
        client_destino = pymongo.MongoClient('mongodb://localhost:27017')
        database_destino = client_destino['bd_temp']
        colecao_destino = database_destino[colecao_temporaria]
        colecao_destino.insert_one(documento)
    except Exception as e:
        logger.error(f'Erro ao salvar documento no banco de dados temporário: {e}')
        

# Ainda não estou usando
def acionar_evolucao (num_documentos, limite_documentos):
    # Verificar se a contagem de documentos atingiu o limite definido pelo usuário
    if num_documentos >= limite_documentos:
        logger.info(f'Atingiu o limite de {limite_documentos} documentos.')
        logger.info(f'Iniciando a atualização com {limite_documentos}')
        
        # Executar a atualização do esquema
        # Usar com id
        # executar_atualizacao_do_esquema (Id)
        update = executar_atualizacao_do_esquema() 
        
        if update:
            logger.info('O esquema foi atualizado ...')
            # Atualizando indice do documentos 
            indice_pa += 1
            num_documentos = 0
            atualizacao_do_esquema_concluida = False
            time.sleep(2)
        else:
            logger.info('Esperando a atualização do esquema terminar...')
            time.sleep(30)
    else:
        atualizacao_do_esquema_concluida = False
        
        
def make_request(url, data, headers):
         
    start_time = time.time()
    response = requests.post(url, json=data, headers=headers)
    elapsed_time = time.time() - start_time
    
    # Imprime as informações
    print(f"URL: {url}")
    print(f"Status code: {response.status_code}")
    
    try:
        # Tenta decodificar a resposta JSON
        response_json = response.json()
        print(f"Response: {response_json}")
    except JSONDecodeError:
        # Se houver um erro ao decodificar, imprime uma mensagem indicando que não foi possível decodificar a resposta
        print("Response: Unable to decode JSON")
    
    print(f"Tempo de processamento: {elapsed_time} segundos")
    print("------------------------")  # Adiciona uma linha para separar as saídas
    
    return response, elapsed_time


def requisicao_batch (collectionName, databaseName, headers, port):
    
    # Configuração da requisição em batch
    elapsed_times_batch = []
    url_batch = 'http://localhost:4200/api/batch/rawschema/steps/all'

    # Dados do lote (batch) para twitter_400
    data_batch = {
        "authentication": {
            "authMechanism": "SCRAM-SHA-1",
            "userName": config['database']['userName'],
            "password": config['database']['password']
        },
        "port": port,
        "address": "localhost",
        "databaseName": databaseName,
        "collectionName": collectionName,
        "userId": "663a47da84467a7a6119ce35",
        "batchId": "65526db0989842e5e5d89e1d"
    }

    response_batch, elapsed_time_batch = make_request(url_batch, data_batch, headers)
    elapsed_times_batch.append(elapsed_time_batch)

    return response_batch, elapsed_time_batch

def requisicao_update(headers, colecao_temporaria, response_batch):
    
    elapsed_times_update = []
    url_update = 'http://localhost:4200/api/batch/rawschema/steps/allupdate'
        
    data_update = {
        "authentication": {
            "authMechanism": "SCRAM-SHA-1",
            "userName": config['database']['userName'],
            "password": config['database']['password']
        },
        "port": "27017",
        "address": "localhost",
        "databaseName": 'bd_temp',
        "collectionName": colecao_temporaria,
        "userId": "655a51b42a775fdad3eab456",
        "batchId": response_batch.json().get('batchId')
    }
    
    try:
        response_update, elapsed_time_update = make_request(url_update, data_update, headers)
        elapsed_times_update.append(elapsed_time_update)
        executar_atualizacao_do_esquema = True
        #return True  # Atualização bem-sucedida
    except Exception as e:
        print(f'Erro durante a atualização do esquema: {e}')
        executar_atualizacao_do_esquema = False
        #return False  # Falha na atualização

    return response_update, elapsed_times_update, executar_atualizacao_do_esquema  

In [ ]:
def main():
    # Conecta-se ao MongoDB
    client = conectar_mongo()
    resume_token = None
    num_documentos = 0
    indice_incremento = 1
    atualizacao_do_esquema_concluida = True

    try:
        response_batch, elapsed_time_update = requisicao_batch (collectionName, 
                                                                databaseName, 
                                                                headers, port)
        while True:
            # monitorando 
            with iniciar_changestream(client, colecao_monitorada, resume_token) as change_stream:
                for change in change_stream:
                    # Extrai o documento inserido e determina a coleção de destino
                    documento_inserido = change['fullDocument']
                    #limite_documentos = primeiro_termo_pa + (indice_pa * razao_pa)
                    
                    
                    # Em vez de usar colecao_ deve usar o nome da coleção que esta sendo monitorada. 
                    colecao_temporaria = f'incremento_{indice_incremento}'
                    
                    # Salva o documento em outro banco de dados
                    salvar_documento_banco_temporario(documento_inserido, colecao_temporaria)
                                     
                    #salvar_resume_token(change['_id'], 'resume_token.json')

                    resume_token = change['_id']
                    num_documentos += 1

                    # logger.info(f'num_documentos: {num_documentos}')
                    # logger.info(dumps(change))
                    # logger.info('')
                    
                    if num_documentos >= limite_documentos:
                        logger.info(f'Atingiu o limite de {limite_documentos} documentos.')
                        logger.info(f'Iniciando a atualização com {limite_documentos} documentos...')
                        
                        # Executa a atualização do esquema
                        response_update, elapsed_times_update, atualizacao_do_esquema_concluida = requisicao_update (
                            headers, 
                            colecao_temporaria,
                            response_batch)
                        if atualizacao_do_esquema_concluida:
                            logger.info('A atualização do esquema terminou... ')
                            # Atualiza o índice e reinicia a contagem de documentos
                            indice_incremento += 1
                            num_documentos = 0
                            atualizacao_do_esquema_concluida = False
                            time.sleep(2)
                        else:
                            logger.info('Esperando a atualização do esquema terminar...')
                            time.sleep(30)
                    else:
                        atualizacao_do_esquema_concluida = False

                # Verifica se a atualização do esquema foi concluída
                if not atualizacao_do_esquema_concluida:
                    logger.info('Atividades de atualização concluídas.')
                    break

    except KeyboardInterrupt:
        logger.info('Encerrando programa devido a KeyboardInterrupt.')
    except pymongo.errors.PyMongoError as e:
        logger.error(f"Erro no ChangeStream: {e}")
    except Exception as e:
        logger.error(f"Erro inesperado: {e}")

    time.sleep(1)

# Ponto de entrada do script
if __name__ == "__main__":
    main()
